# Redes Neurais

Redes neurais são formadas por nós organizados em camadas. Em uma rede neural **densa**, todos os nós de uma dada camada estão conectados a todos os nós das camadas anterior e posterior:

![](neural_net.svg)
*<center>crédito: victorzhou.com</center>*

A <font color='blue'>primeira camada</font> é composta pelo vetor de entrada $X$, que também podemos chamar de *feature vector*. A <font color='green'>última camada</font> é composta pelo vetor de saída $y$. As camadas intermediárias são chamadas de camadas ocultas. Uma rede neural com mais de uma camada oculta é chamada de rede neural **profunda**. Daí vem o tão conhecido termo *deep learning*.

Cada nó faz uma combinação linear de todas as entradas que recebe e depois aplica uma transformação não-linear $\sigma$ a esta combinação:

$z_{nó} = \sigma(w_1 \cdot x_1 + w_2 \cdot x_2 + ... w_n \cdot x_n + b)$

Em notação vetorial, para uma dada camada, seu vetor de saída $Z$ é:

$Z = \sigma(W \cdot X + B)$

As matrizes de pesos $W$ são os parâmetros que queremos otimizar. A transformação não-linear $\sigma$ é chamada de **função de ativação**.

Podemos usar `keras`, uma biblioteca de alto nível, para construir uma rede como a do desenho acima. Chamando o método `.summary()` no modelo, podemos ver a quantidade de parâmetros dele.

In [1]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential

x = Input(shape=(4,))

z = Dense(6, activation='relu')(x)
z = Dense(6, activation='relu')(z)

y = Dense(2, activation='softmax')(z)

model = Model(inputs=x, outputs=y)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 30        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 14        
Total params: 86
Trainable params: 86
Non-trainable params: 0
_________________________________________________________________


### Classificando dígitos com o MNIST

In [2]:
from tensorflow.keras import datasets
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D

def compile_train(model, X_train, y_train):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    model.fit(
        X_train,
        y_train,
        verbose=1,
        epochs=5,
        batch_size=32,
        validation_split=0.1,
    )

In [3]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

print('dataset shape', X_train.shape)

X_train = X_train/255
X_test = X_test/255

X_train = X_train.reshape((-1, 784))
X_test = X_test.reshape((-1, 784))

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(784,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

compile_train(model, X_train, y_train)

dataset shape (60000, 28, 28)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________
Train on 54000 samples, validate on 6000 samples
Epoch 1/5
   32/54000 [..............................] - ETA: 7:03

InternalError:  Blas GEMM launch failed : a.shape=(32, 784), b.shape=(784, 64), m=32, n=64, k=784
	 [[node sequential/dense_3/MatMul (defined at <ipython-input-2-2d343de8733f>:13) ]] [Op:__inference_distributed_function_816]

Function call stack:
distributed_function


### Classificando peças de roupa com o Fashion MNIST

Esta é uma variante um pouco mais desafiadora do MNIST: é um dataset composto por 10 tipos de peça de roupa.

In [4]:
(X_train, y_train), (X_test, y_test) = datasets.fashion_mnist.load_data()

X_train = X_train/255
X_test = X_test/255

X_train = X_train.reshape((-1, 784))
X_test = X_test.reshape((-1, 784))

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(784,)))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

compile_train(model, X_train, y_train)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 3s 65us/sample - loss: 0.4977 - acc: 0.8208 - val_loss: 0.4089 - val_acc: 0.8492
Epoch 2/5
54000/54000 [==============================] - 3s 63us/sample - loss: 0.3686 - acc: 0.8659 - val_loss: 0.3934 - val_acc: 0.8575
Epoch 3/5
54000/54000 [==============================

## Redes Neurais Convolucionais

Numa rede neural densa, a quantidade de conexões (e, consequentemente, de parâmetros) cresce *exponencialmente* ao adicionarmos mais nós. Assim, para dados de dimensão alta, ou para problemas que exigem modelos muito complexos, é preciso buscar estratégias de conectividade mais eficientes. No caso de imagens, usamos **convoluções**.

Numa camada convolucional, também são feitas combinações lineares da camada anterior. A diferença é que agora cada saída é resultado da combinação linear de um subconjunto da entrada, geralmente definido por uma janela quadrada. Isso reduz bastante a quantidade de parâmetros e facilita o reconhecimento de *padrões locais* em imagens.

![](conv.gif)

*<center>crédito: https://github.com/vdumoulin/conv_arithmetic</center>*

Vamos construir uma rede neural usando convoluções para tentar melhorar nosso classificador do Fashion MNIST.

In [5]:
(X_train, y_train), (X_test, y_test) = datasets.fashion_mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

X_train = X_train/255
X_test = X_test/255

model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), input_shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=(3,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

compile_train(model, X_train, y_train)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        18464     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                46090     
Total params: 65,194
Trainable params: 65,194
Non-trainable params: 0
_________________________________________________________________
Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 6s 111us/s